In [ ]:
import plotly_express as px
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import mplfinance as mpf
import plotly.graph_objects as go
import yfinance as yf
from backtesting import Backtest, Strategy
import pandas_ta as ta

from backtesting.lib import crossover
import math

In [ ]:
# If you want to check if the strategy is executing trades correctly, use this to validate the data that can be run with the backtest and can handle the plot.
ticker = "usdjpy=X"
df = yf.download(ticker, period="1y", interval="1h")


In [ ]:
# For other spreadsheet use this in reading csv
# , delimiter='\t', names=['Open', 'High', 'Low', 'Close', 'Volume'], header=0

# Darwinex spreadsheet
df = pd.read_csv('../Data/Darwinex/GBPJPY60.csv', header=None, names=['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume'])
df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
df.set_index('DateTime', inplace=True)
df.drop(columns=['Date', 'Time'], inplace=True)


In [ ]:
df = df.iloc[:30]

In [ ]:
# fig = go.Figure(data=[go.Candlestick(x=df.index,
#                 open=df.Open,
#                 high=df.High,
#                 low=df.Low,
#                 close=df.Close)])

# fig.update_layout(
#     xaxis=dict(type='category', nticks=20),
#     xaxis_rangeslider_visible = False
# )

# fig.show()


In [58]:
# For other spreadsheet use this in reading csv
# , delimiter='\t', names=['Open', 'High', 'Low', 'Close', 'Volume'], header=0

# Darwinex spreadsheet
df = pd.read_csv('../Data/Darwinex/GBPJPY60.csv', header=None, names=['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume'])
df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
df.set_index('DateTime', inplace=True)
df.drop(columns=['Date', 'Time'], inplace=True)

df = df.iloc[:150]

# Init
conditions = [
    df.Close > df.Open,
    df.Close < df.Open
]

choices = ['Bull', 'Bear']

df['Candle'] = np.select(conditions, choices, default='Doji')
df['Swing_Low'] = None
df['Swing_High'] = None
df['Swing_High'] = None
df['Swing_Low'] = None
df['Trend'] = 'Neutral'

higher_high = float('-inf')
lower_low = float('inf')

bullish_pullback_count = 0
bearish_pullback_count = 0
bullish_pullback = False
bearish_pullback = False

swing_high = df['High'].iloc[0]
swing_low = df['Low'].iloc[0]

uptrend_lowest_pullback = float('inf')
downtrend_highest_pullback = float('inf')

previous_high = df['High'].iloc[0]
previous_low = df['Low'].iloc[0]

current_trend = 'Neutral'
df.at[0, 'Trend'] = current_trend

df['Price_Status'] = None
price_status = None




In [59]:
from IPython.display import clear_output
previous_row = df.iloc[0]
# Itterating logic starts
for i, row in df.iloc[1:].iterrows():
    current_high = row['High']
    current_low = row['Low']
    candle_type = row['Candle']
    current_close = row['Close']
    
    # clear_output(wait=True)
    
    if current_close > swing_high:
        current_trend = 'Uptrend'

    elif current_close < swing_low:
        current_trend = 'Downtrend'
    
    df.at[i, 'Trend'] = current_trend

    if bullish_pullback or bearish_pullback:
        price_status = 'Consolidation'
    else:
        price_status = 'Extension'

    df.at[i, 'Price_Status'] = price_status
    # ================================================================================================== #


    if current_trend == 'Uptrend':
        if current_high > previous_high and not bullish_pullback:            
            higher_high = current_high
        elif current_close < higher_high and candle_type == 'Bear':
            bullish_pullback_count += 1
        
        if bullish_pullback_count >= 2:
            bullish_pullback = True
        else:
            bullish_pullback = False

        if bullish_pullback and current_close > swing_low:
             swing_high = higher_high
             df.at[i, 'Swing_High'] = swing_high
             if current_low < previous_low:
                  uptrend_lowest_pullback = current_low                
        else:
             if current_close > swing_high:
                  swing_low = uptrend_lowest_pullback
                  df.at[i, 'Swing_Low'] = swing_low
    
    if current_trend == 'Uptrend':
        if current_high > previous_high and not bullish_pullback:            
            higher_high = current_high
              


In [60]:
df.head(23)

,Open,High,Low,Close,Volume,Candle,Swing_Low,Swing_High,Trend,Price_Status
DateTime,,,,,,,,,,
2013-08-21 09:00:00,152.669,152.850,152.573,152.823,4543.0,Bull,None,None,Neutral,None
2013-08-21 10:00:00,152.824,152.947,152.419,152.864,5086.0,Bull,inf,None,Uptrend,Extension
2013-08-21 11:00:00,152.872,152.991,152.706,152.829,3732.0,Bear,None,inf,Downtrend,Extension
2013-08-21 12:00:00,152.828,152.961,152.683,152.713,3534.0,Bear,None,inf,Downtrend,Extension
2013-08-21 13:00:00,152.712,153.005,152.646,152.956,3639.0,Bull,None,inf,Downtrend,Extension
2013-08-21 14:00:00,152.958,153.058,152.925,152.952,3292.0,Bear,None,inf,Downtrend,Extension
2013-08-21 15:00:00,152.952,153.253,152.940,153.166,4806.0,Bull,None,inf,Downtrend,Extension
2013-08-21 16:00:00,153.169,153.321,153.035,153.219,4699.0,Bull,None,inf,Downtrend,Extension
2013-08-21 17:00:00,153.219,153.262,153.028,153.098,3712.0,Bear,None,inf,Downtrend,Extension


In [61]:
fig = go.Figure(data=[go.Candlestick(x=df.index,
                                     open=df['Open'],
                                     high=df['High'],
                                     low=df['Low'],
                                     close=df['Close'],
                                     name='Candlestick')])

# Add trend markers (prioritize uptrend over downtrend)
uptrend_markers = df[df['Trend'] == 'Uptrend']
downtrend_markers = df[df['Trend'] == 'Downtrend']

# Filter out downtrend markers that coincide with uptrend markers
downtrend_markers_filtered = downtrend_markers[~downtrend_markers.index.isin(uptrend_markers.index)]

fig.add_trace(go.Scatter(x=uptrend_markers.index, y=uptrend_markers['Low'] - 0.001 * uptrend_markers['Low'], mode='markers', name='Uptrend', marker=dict(symbol='triangle-up', color='blue', size=8)))
fig.add_trace(go.Scatter(x=downtrend_markers_filtered.index, y=downtrend_markers_filtered['High'] + 0.001 * downtrend_markers_filtered['High'], mode='markers', name='Downtrend', marker=dict(symbol='triangle-down', color='orange', size=8)))

# Update the layout for a clearer view
fig.update_layout(title='Candlestick Chart with Swing Highs and Lows',
                  height=600,
                  template='plotly_dark',
                  xaxis=dict(type='category', nticks=20, showgrid=False),
                  xaxis_rangeslider_visible=False)

# Show the figure
fig.show()

In [ ]:
# Split the values in the 'Trend' column into a list of trends and check if there is more than one trend in each row
if (df['Trend'].str.split(',').apply(lambda x: len(x)) > 1).any():
    print("There is more than one trend listed in at least one row of the 'Trend' column.")
else:
    print("Each row of the 'Trend' column contains only one trend.")
